In [1]:
import warnings
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")# to ignore warning 


In [2]:
import matplotlib as plt
import seaborn as sns

In [3]:
import os
audio_folder = "/Users/chhavigupta/Downloads/voice_samples"
audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]
print(audio_files)

['hindi_0597.wav', 'hindi_0554.wav', 'hindi_0232.wav', 'hindi_0226.wav', 'hindi_1138.wav', 'hindi_0540.wav', 'hindi_0795.wav', 'hindi_0959.wav', 'hindi_0781.wav', 'hindi_0971.wav', 'hindi_0965.wav', 'hindi_0030.wav', 'hindi_0756.wav', 'hindi_0742.wav', 'hindi_0024.wav', 'hindi_0018.wav', 'hindi_0839.wav', 'recording2020_01_16_11_03_51.wav', 'arctic_a0387.wav', 'hindi_0805.wav', 'hindi_0811.wav', 'hindi_0622.wav', 'hindi_0636.wav', 'hindi_0150.wav', 'hindi_0178.wav', 'hindi_0385.wav', 'hindi_0391.wav', 'hindi_0420.wav', 'hindi_0346.wav', 'hindi_0352.wav', 'hindi_0434.wav', 'hindi_1070.wav', 'hindi_0409.wav', 'hindi_0353.wav', 'hindi_0421.wav', 'hindi_0390.wav', 'hindi_0384.wav', 'hindi_0179.wav', 'hindi_0637.wav', 'hindi_0151.wav', 'hindi_0623.wav', 'hindi_0810.wav', 'hindi_0804.wav', 'hindi_0838.wav', 'hindi_0025.wav', 'hindi_0031.wav', 'hindi_0757.wav', 'hindi_0964.wav', 'usfemale (15).wav', 'hindi_0970.wav', 'hindi_0780.wav', 'hindi_0958.wav', 'hindi_0794.wav', 'hindi_1105.wav', 'hin

In [4]:
import whisper

model = whisper.load_model("tiny")#load_model("base") will load pre trained whisper model base and also base has better accuracy then tiny

def transcribe_audio(file_path):
    result = model.transcribe(file_path)  # Transcribes the audio file
    return result['text']  # Extracts only the text (transcript)


In [5]:
import language_tool_python #is used to check spelling ,grammar and style in  english 
tool = language_tool_python.LanguageTool('en-US')
def get_grammar_score(text):
    matches = tool.check(text)#finds grammar/spelling issues
    num_errors = len(matches)#how many grammar issues
    words = len(text.split())#how many words in a text
    if words == 0:                      
        return 0, num_errors, matches
    score = max(0, 100 - (num_errors / words) * 100)#calculate grammar score 0 to 100
    return round(score,2),num_errors,matches

In [ ]:
results = []

for file in audio_files:
    path = os.path.join(audio_folder, file)  
    transcript = transcribe_audio(path)
    score, errors, matches = get_grammar_score(transcript)

    results.append({
        "file": file,
        "transcript": transcript,
        "grammar_score": score,
        "errors": errors
    })


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.sort_values(by="grammar_score",inplace=True)#inplace = True is used to directly modify dataframe without making copy of it
df.head()

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))#10 width and 6 height
sns.barplot(x="grammar_score", y="file", data=df)
plt.title("Grammar Scores for Audio Samples")
plt.xlabel("Grammar Score")
plt.ylabel("Audio File")
plt.show()